In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/pakistan-car-prices/updated_pakwheels.csv')

# 1. Preprocessing

In [ ]:
df.head()

In [ ]:
df.dtypes

# 1) Add Brand Feature

In [ ]:
df['Brand']=df.Name.apply(lambda x: x.split()[0])

# 2) Break down 'Features' into each 'Functions'

In [ ]:
df['Features'].unique()

In [ ]:
df=df.fillna({'Features': '0'})

In [ ]:
df1=pd.DataFrame(df['Features'].str.strip("'").str.split(',',expand=True).stack())
df1=df1.rename(columns={0: 'func'})
df1=df1.reset_index()

In [ ]:
df_func=pd.DataFrame(df1.groupby(['level_0','func'])['func'].count())

In [ ]:
df_func=pd.DataFrame(df_func.unstack())
df_func

In [ ]:
df_func.describe()

In [ ]:
df_func.columns

In [ ]:
def rename_multicol(df_func):
    df_col=df_func.columns
    df_func = df_func.T.reset_index(drop=False).T
    for  i in range(df_func.shape[1]): 
        rename_col = {i:"".join(df_col[i])}
        df_func = df_func.rename(columns = rename_col)     
    df_func = df_func.drop(["level_0"],axis=0)
    return df_func

df_func=rename_multicol(df_func)

In [ ]:
df_func

In [ ]:
df_func=df_func.drop(df_func.index[[0]])

In [ ]:
df_func=df_func.drop('func0', axis=1)

In [ ]:
df_func

# 3) Choose the features and create new data set

In [ ]:
df_data=df.loc[:,['Price','Model Year','Mileage','Engine Type','Engine Capacity','Transmission','Color','Assembly','Body Type','Brand']]

In [ ]:
df_data[['Engine Type','Transmission','Body Type']] = df_data[['Engine Type','Transmission','Body Type']].astype(str)

In [ ]:
df_data

In [ ]:
df_data.dtypes

In [ ]:
df_data=pd.concat([df_data, df_func], axis=1)

In [ ]:
df_data

In [ ]:
df_data=pd.get_dummies(df_data)

In [ ]:
df_data

In [ ]:
df_data.dtypes

In [ ]:
df_data.isnull().sum()

In [ ]:
df_data.columns

# 2. Modeling for prediction by Pycaret

In [ ]:
pip install pycaret

In [ ]:
from pycaret.regression import *

In [ ]:
exp1 = setup(df_data, target = 'Price', ignore_features = None,silent=True)

# 1) Choose the best model

In [ ]:
compare_models()

# 2) Modeling by catboost and tuning

In [ ]:
cb = create_model('catboost')

In [ ]:
tuned_cb = tune_model(cb)

# 3) Model visualization

In [ ]:
plot_model(tuned_cb)

# 4) Model understanding

In [ ]:
interpret_model(tuned_cb)

# 5) Prediction model

In [ ]:
predictions = predict_model(cb, data=df_data)

In [ ]:
predictions